In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import time

from tqdm            import tqdm

from propagation import received_power, quad_distribute_power, propagation_delay, \
                        map_rx_config, map_tx_config

from parameter_measurement import measure_xR_roberts, measure_DB_soner, generate_clocks

In [2]:
weather     = 'clear' 
temperature = 298 # Kelvin
daynight    = 'night'


daynight_noise_factors = dict()
daynight_noise_factors['day_directsun']   = 1.000 # 5100 uA
daynight_noise_factors['day_indirectsun'] = 0.145 # 740 uA
daynight_noise_factors['night']           = 0.010 # very small

# minmax bounds to be safe if you set the noise factor yourself
bg_current = (np.minimum(1, np.maximum(0, daynight_noise_factors[daynight]))*5100)*1e-6;

weather_attenuation_factors = dict()
weather_attenuation_factors['clear'] = 0.0
weather_attenuation_factors['rain']  = -0.05
weather_attenuation_factors['fog']   = -0.2

atten = weather_attenuation_factors[weather];

### loading

In [3]:
L = 1.6 # vehicle width in meters, check below

yRL_to_rxL = np.linspace(5.0,20.0,4);
yRL_to_rxL = np.linspace(5.0,20.0,4);


xRL_to_rxL = a['x_RL'][0:trajectory_partial_numsamples];  
xRR_to_rxL = a['x_RR'][0:trajectory_partial_numsamples]; yRR_to_rxL = a['y_RR'][0:trajectory_partial_numsamples]
yRL_to_rxR = a['y_RL'][0:trajectory_partial_numsamples]; yRR_to_rxR = a['y_RR'][0:trajectory_partial_numsamples]; 
hdg  = a['hdg'][0:trajectory_partial_numsamples];
xRL_to_rxR = a['x_RL'][0:trajectory_partial_numsamples] - L;
xRR_to_rxR = a['x_RR'][0:trajectory_partial_numsamples] - L;

In [4]:
a = np.load('optics/qrx_planoconvex.npz');
f_QRX, pd_snst, pd_gain, pd_dim, rx_P_rx_factor, rx_I_bg_factor, rx_thermal_factor1, rx_thermal_factor2 = map_rx_config(a);

In [5]:
a = np.load('optics/tx_lambertian_20deg_2W.npz')
tx_ha, tx_pwr, tx_norm, tx_lambertian_order = map_tx_config(a);

### computing

In [6]:
pwr_txL_to_rxL = received_power(xRL_to_rxL, yRL_to_rxL, 0, pd_dim/1000, tx_pwr, tx_norm, tx_lambertian_order, atten)
pwr_txL_to_rxR = received_power(xRL_to_rxR, yRL_to_rxR, 0, pd_dim/1000, tx_pwr, tx_norm, tx_lambertian_order, atten)
pwr_txR_to_rxL = received_power(xRR_to_rxL, yRR_to_rxL, 0, pd_dim/1000, tx_pwr, tx_norm, tx_lambertian_order, atten)
pwr_txR_to_rxR = received_power(xRR_to_rxR, yRR_to_rxR, 0, pd_dim/1000, tx_pwr, tx_norm, tx_lambertian_order, atten)

In [8]:
c         = 299792458  # [m/s] speed of light
f_adc_clk = 1e8;       # [Hz], measurement clock freq
f_e       = 1.0e6;     # [Hz], emitted wave freq, left TX
r         = 4999;      # unitless, heterodyning factor
N         = 4;         # unitless, averaging factor for left TX

f_sim = 1e10 # [Hz]

t_sim, t_sim_stop, spatial_sim_resolution, s_sim, s_adc_clk_re = generate_clocks(c, f_e, N, r, f_sim, f_adc_clk)

print("sim stop time:",t_sim_stop, "s")
print("sim length   :",int(t_sim_stop/t_sim),"samples")
print("sim time res :", t_sim*(10**12), "ps")
print("sim tvec size:", s_sim.nbytes/(1024**3), "GB")
print("sim space res:", 100*spatial_sim_resolution, "cm")

sim stop time: 0.0105 s
sim length   : 105000000 samples
sim time res : 100.0 ps
sim tvec size: 0.782310962677002 GB
sim space res: 2.99792458 cm
